In [12]:
# Import necessary libraries
import pandas as pd
import glob
from ydata_profiling import ProfileReport #pandas_profiling
import numpy as np
import json
import re
import duckdb
import functions_project

In [13]:
# Read imdb validation set
validation_df = pd.read_csv('data/validation_hidden.csv', index_col = 1)
validation_df = validation_df.drop(columns=['Unnamed: 0'])

# Replace all missing values by legit missing values instead of string "\N"
validation_df = validation_df.replace('\\N', np.nan).reset_index()

# Replace startYear NA with the value of endYear
validation_df['startYear']=validation_df['startYear'].fillna(validation_df['endYear'])

validation_df.head()

,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes
0,tt0003740,Cabiria,NaN,1914,NaN,148,3452.0
1,tt0008663,A Man There Was,Terje Vigen,1917,NaN,65,1882.0
2,tt0010307,J'accuse!,NaN,1919,NaN,166,1692.0
3,tt0014429,Safety Last!,Safety Last!,1923,NaN,74,19898.0
4,tt0015175,Die Nibelungen: Siegfried,NaN,1924,NaN,143,5676.0


In [14]:
# Read imdb test set
test_df = pd.read_csv('data/test_hidden.csv', index_col = 1)
test_df = test_df.drop(columns=['Unnamed: 0'])

# Replace all missing values by legit missing values instead of string "\N"
test_df = test_df.replace('\\N', np.nan).reset_index()

# Replace startYear NA with the value of endYear
test_df['startYear']=test_df['startYear'].fillna(test_df['endYear'])

test_df.head()

,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes
0,tt0014972,He Who Gets Slapped,He Who Gets Slapped,1924,NaN,95,3654.0
1,tt0015016,The Iron Horse,NaN,1924,NaN,150,2136.0
2,tt0015174,Die Nibelungen: Kriemhild's Revenge,NaN,1924,NaN,129,4341.0
3,tt0015214,At 3:25,NaN,1925,1925,59,1724.0
4,tt0015863,Go West,NaN,1925,NaN,69,4188.0


In [15]:
#call function map_cleaned_strings to apply in the dataframe on the column primary title
functions_project.map_cleaned_strings(validation_df, 'primaryTitle')
functions_project.map_cleaned_strings(test_df, 'primaryTitle')


0                     he who gets slapped
1                          the iron horse
2       die nibelungen kriemhilds revenge
3                                  at 325
4                                 go west
                      ...                
1081                   one piece stampede
1082                         the big ugly
1083                           pagalpanti
1084                the wedding unplanner
1085            android kunjappan ver 525
Name: primaryTitle, Length: 1086, dtype: object

In [16]:
# Save the tables in the db and stop the connection afterwards
con = duckdb.connect('imdb_reviews.db')
con.sql("CREATE TABLE IF NOT EXISTS imdb_review_validation AS SELECT * FROM validation_df")
con.sql("CREATE TABLE IF NOT EXISTS imdb_review_test AS SELECT * FROM test_df")
con.close()

In [17]:
con.close()
